In [1]:
from resources import *
import pandas as pd

Initializing resources...
... packages imported!
... resources initialized!


# Part 1: glider data

In [2]:
gliderData = pd.read_csv('turbulence/data/gliderExportThesis.csv')

topSalt = gliderData['topSaltConvolved']
bottomSalt = gliderData['bottomSaltConvolved']
topTemp = gliderData['topTempConvolved']
bottomTemp = gliderData['bottomTempConvolved']

topSalt = (topSalt - np.min(topSalt)) / (np.max(topSalt) - np.min(topSalt))
bottomSalt = (bottomSalt - np.min(bottomSalt)) / (np.max(bottomSalt) - np.min(bottomSalt))
topTemp = (topTemp - np.min(topTemp)) / (np.max(topTemp) - np.min(topTemp))
bottomTemp = (bottomTemp - np.min(bottomTemp)) / (np.max(bottomTemp) - np.min(bottomTemp))

In [3]:
def interp(data, xNew):
    xOld = np.linspace(0, 1, len(data))
    yOld = data
    xNew = np.linspace(0, 1, xNew)
    yNew = np.interp(xNew, xOld, yOld)
    return yNew

interpLen = 2 * 60 * SAMPLE_RATE
topSaltInterp = interp(topSalt, interpLen)
bottomSaltInterp = interp(bottomSalt, interpLen)
topTempInterp = interp(topTemp, interpLen)
bottomTempInterp = interp(bottomTemp, interpLen)

In [4]:
seconds = 2 * 60

In [ ]:
def vol_oscillator(rate, phase):
    oscillator = lfo(rate, seconds, phase)
    return oscillator


# Bottom drone
fundamentalPitch = notename_to_hertz('E-2')
sine_low = 2 * sineWave(fundamentalPitch / 2, seconds)
sine_fundamental = 2 * sineWave(fundamentalPitch, seconds)

sineVol = 0.4 + 0.05 * vol_oscillator(0.1, np.pi/6)
sineFifth = sineVol * sineWave(1.5 * fundamentalPitch - 0.4, seconds) + sineVol * sineWave(1.5 * fundamentalPitch + 0.4, seconds)
sineFifth = sineFifth * (15 - 5 * bottomSaltInterp)

sawtoothVol = 0.4 - 0.05 * vol_oscillator(0.09, np.pi/3)
sawtoothFifth = sawtoothVol * sawtoothWave(1.5 * fundamentalPitch - 0.6, seconds) + sawtoothVol * sawtoothWave(1.5 * fundamentalPitch + 0.6, seconds)
sawtoothFifth = sawtoothFifth * (1 + 10 * bottomSaltInterp)

sineVol = 0.4 + 0.05 * vol_oscillator(0.08, np.pi / 2)
sineOctave = sineVol * sineWave(2 * fundamentalPitch - 0.8, seconds) + sineVol * sineWave(2 * fundamentalPitch + 0.8, seconds)
sineOctave = sineOctave * (15 - 5 * bottomSaltInterp)

sawtoothVol = 0.4 + 0.05 * vol_oscillator(0.07, 2 * np.pi / 3)
sawtoothOctave = sawtoothVol * sawtoothWave(2 * fundamentalPitch - 1, seconds) + sawtoothVol * sawtoothWave(2 * fundamentalPitch + 1, seconds)
sawtoothOctave = sawtoothOctave * (1 + 10 * bottomSaltInterp)

sumL = sine_low + sine_fundamental + 0.6 * sineFifth + 0.4 * sawtoothFifth + 0.1 * sineOctave + 0.9 * sawtoothOctave
sumR = sine_low + sine_fundamental + 0.4 * sineFifth + 0.6 * sawtoothFifth + 0.9 * sineOctave + 0.1 * sawtoothOctave

sumLEffects = (1 + bottomTempInterp) * sumL
sumREffects = (1 + bottomTempInterp) * sumR

stereo_sound_bottom = np.column_stack((sumLEffects, sumREffects))

#write('turbulence/stereo_drone_bottom.wav', SAMPLE_RATE, stereo_sound_bottom)

In [7]:
# Top drone

fundamentalPitch = notename_to_hertz('E-4')
sine_low = 2 * sineWave(fundamentalPitch / 2, seconds)
sine_fundamental = 2 * sineWave(fundamentalPitch, seconds)

sineVol = 0.4 + 0.05 * vol_oscillator(0.1, np.pi/6)
sineFifth = sineVol * sineWave(1.5 * fundamentalPitch - 0.3, seconds) + sineVol * sineWave(1.5 * fundamentalPitch + 0.3, seconds)
sineFifth = sineFifth * (15 - 5 * topSaltInterp)

sawtoothVol = 0.4 - 0.05 * vol_oscillator(0.09, np.pi/3)
sawtoothFifth = sawtoothVol * sawtoothWave(1.5 * fundamentalPitch - 0.5, seconds) + sawtoothVol * sawtoothWave(1.5 * fundamentalPitch + 0.5, seconds)
sawtoothFifth = sawtoothFifth * (1 + 10 * topSaltInterp)

sineVol = 0.4 + 0.05 * vol_oscillator(0.08, np.pi / 2)
sineOctave = sineVol * sineWave(2 * fundamentalPitch - 0.7, seconds) + sineVol * sineWave(2 * fundamentalPitch + 0.7, seconds)
sineOctave = sineOctave * (15 - 5 * topSaltInterp)

sawtoothVol = 0.4 + 0.05 * vol_oscillator(0.07, 2 * np.pi / 3)
sawtoothOctave = sawtoothVol * sawtoothWave(2 * fundamentalPitch - 0.9, seconds) + sawtoothVol * sawtoothWave(2 * fundamentalPitch + 0.9, seconds)
sawtoothOctave = sawtoothOctave * (1 + 10 * topSaltInterp)

sumL = sine_low + sine_fundamental + 0.6 * sineFifth + 0.4 * sawtoothFifth + 0.1 * sineOctave + 0.9 * sawtoothOctave
sumR = sine_low + sine_fundamental + 0.4 * sineFifth + 0.6 * sawtoothFifth + 0.9 * sineOctave + 0.1 * sawtoothOctave

sumLEffects = (1 + topTempInterp) * sumL
sumREffects = (1 + topTempInterp) * sumR

stereo_sound_top = np.column_stack((sumLEffects, sumREffects))

#write('turbulence/stereo_drone_high.wav', SAMPLE_RATE, stereo_sound_top)


In [8]:
turbulence_pt1 = stereo_sound_top + stereo_sound_bottom
#write('turbulence/turbulence_pt1.wav', SAMPLE_RATE, turbulence_pt1)

# Part 2: Satellite data

In [10]:
eddyData = pd.read_csv('turbulence/data/dfFocusExport.csv')

In [11]:
seconds = 2 * 60
amp = eddyData['amplitude']
speed = eddyData['speed']

amp = (amp - np.min(amp)) / (np.max(amp) - np.min(amp))
speed = (speed - np.min(speed)) / (np.max(speed) - np.min(speed))

ampInterp = interp(amp, 2 * 60 * SAMPLE_RATE)
speedInterp = interp(speed, 2 * 60 * SAMPLE_RATE)
fundamentalPitch = notename_to_hertz('A-2')

In [ ]:
sine_low = 2 * sineWave(fundamentalPitch / 2, seconds)
sine_fundamental = 2 * sineWave(fundamentalPitch, seconds)

squareVol = 0.4 + 0.05 * vol_oscillator(0.1, np.pi/6) #np.sin(np.pi * volx / SAMPLE_RATE + np.pi/5)
squareFifthDown = squareVol * squareWave(1.5 * fundamentalPitch - 1, seconds)
squareFifthUp = squareVol * squareWave(1.5 * fundamentalPitch + 1, seconds)
squareFifth = squareFifthDown + squareFifthUp

sawtoothVol = 0.4 - 0.05 * vol_oscillator(0.07, np.pi/3) #np.sin(np.pi * volx / SAMPLE_RATE + 2 * np.pi/5)
sawtoothFifthDown = sawtoothVol * sawtoothWave(1.5 * fundamentalPitch - 0.9, seconds)
sawtoothFifthUp = sawtoothVol * sawtoothWave(1.5 * fundamentalPitch + 0.9, seconds)
sawtoothFifth = sawtoothFifthDown + sawtoothFifthUp

sineVol = 0.3 - 0.05 *  vol_oscillator(0.06, np.pi/2) #np.sin(np.pi * volx / SAMPLE_RATE + 2 * np.pi/5)
sineFifthDown = sineVol * sineWave(1.5 * fundamentalPitch - 0.9, seconds)
sineFifthUp = sineVol * sineWave(1.5 * fundamentalPitch + 0.9, seconds)
sineFifth = sineFifthDown + sineFifthUp

squareVol = 0.4 - 0.05 * vol_oscillator(0.09, np.pi) #np.sin(np.pi * volx / SAMPLE_RATE + 4 * np.pi/5)
squareOctaveDown = squareVol * squareWave(2 * fundamentalPitch - 0.8, seconds)
squareOctaveUp = squareVol * squareWave(2 * fundamentalPitch + 0.8, seconds)
squareOctave = squareOctaveDown + squareOctaveUp

sawtoothVol = 0.4 + 0.05 * vol_oscillator(0.05, 2 * np.pi/3) #np.sin(np.pi * volx / SAMPLE_RATE + 3 * np.pi/5)
sawtoothOctaveDown = sawtoothVol * sawtoothWave(2 * fundamentalPitch - 0.7, seconds)
sawtoothOctaveUp = sawtoothVol * sawtoothWave(2 * fundamentalPitch + 0.7, seconds)
sawtoothOctave = sawtoothOctaveDown + sawtoothOctaveUp

sineVol = 0.3 - 0.05 * vol_oscillator(0.06, 5 * np.pi/6) #np.sin(np.pi * volx / SAMPLE_RATE + 2 * np.pi/5)
sineOctaveDown = sineVol * sineWave(2 * fundamentalPitch - 0.5, seconds)
sineOctaveUp = sineVol * sineWave(2 * fundamentalPitch + 0.5, seconds)
sineOctave = sineOctaveDown + sineOctaveUp

sumL = sine_low + sine_fundamental + 0.6 * squareFifth + 0.4 * sawtoothFifth + 0.7 * sineFifth + 0.1 * squareOctave + 0.9 * sawtoothOctave + 0.3 * sineOctave
sumR = sine_low + sine_fundamental + 0.4 * squareFifth + 0.6 * sawtoothFifth + 0.3 * sineFifth + 0.9 * squareOctave + 0.1 * sawtoothOctave + 0.7 * sineOctave


vol = 5 * ampInterp
sumBoth = vol * (sumL + sumR)
turbulence_pt2 = np.column_stack((vol * sumL, vol * sumR))

#write('turbulence/turbulence_pt2.wav', SAMPLE_RATE, turbulence_pt2)

In [14]:
vel1 = np.mean(speed[ : int(len(speed) / 6)])
vel2 = np.mean(speed[1 * int(len(speed) / 6) : 2 * int(len(speed) / 6)])
vel3 = np.mean(speed[2 * int(len(speed) / 6) : 3 * int(len(speed) / 6)])
vel4 = np.mean(speed[3 * int(len(speed) / 6) : 4 * int(len(speed) / 6)])
vel5 = np.mean(speed[4 * int(len(speed) / 6) : 5 * int(len(speed) / 6)])
vel6 = np.mean(speed[5 * int(len(speed) / 6) : ])

vel1, vel2, vel3, vel4, vel5, vel6

(0.8540132644002296,
 0.749821574668825,
 0.7772599091336362,
 0.5363160826500948,
 0.3214180026807318,
 0.03663514104075793)

In [15]:
sumBoth = sumL + sumR

lfo1 = (lfo(vel1, 17 / vel1) + 1) / 2
eddy1_L = sumBoth[ : len(lfo1)] * lfo1
eddy1_R = sumBoth[ : len(lfo1)] * (1 - lfo1)

lfo2 = (lfo(vel2, 14 / vel2) + 1) / 2
eddy2_L = sumBoth[len(lfo1) : len(lfo1) + len(lfo2)] * lfo2
eddy2_R = sumBoth[len(lfo1) : len(lfo1) + len(lfo2)] * (1 - lfo2)

lfo3 = (lfo(vel3, 15 / vel3) + 1) / 2
eddy3_L = sumBoth[len(lfo2) : len(lfo2) + len(lfo3)] * lfo3
eddy3_R = sumBoth[len(lfo2) : len(lfo2) + len(lfo3)] * (1 - lfo3)

lfo4 = (lfo(vel4, 10 / vel4) + 1) / 2
eddy4_L = sumBoth[len(lfo3) : len(lfo3) + len(lfo4)] * lfo4
eddy4_R = sumBoth[len(lfo3) : len(lfo3) + len(lfo4)] * (1 - lfo4)

lfo5 = (lfo(vel5, 6 / vel5) + 1) / 2
eddy5_L = sumBoth[len(lfo4) : len(lfo4) + len(lfo5)] * lfo5
eddy5_R = sumBoth[len(lfo4) : len(lfo4) + len(lfo5)] * (1 - lfo5)

lfo6 = (lfo(vel6, 20) + 1) / 2
eddy6_L = sumBoth[len(lfo5) : len(lfo5) + len(lfo6)] * lfo6
eddy6_R = sumBoth[len(lfo5) : len(lfo5) + len(lfo6)] * (1 - lfo6)

In [16]:
eddy_L = np.concatenate([eddy1_L, eddy2_L, eddy3_L, eddy4_L, eddy5_L, eddy6_L])
eddy_R = np.concatenate([eddy1_R, eddy2_R, eddy3_R, eddy4_R, eddy5_R, eddy6_R])


In [17]:
len(eddy_L), len(ampInterp)

(5079819, 5292000)

In [18]:
eddy_L = eddy_L * ampInterp[:len(eddy_L)]
eddy_R = eddy_R * ampInterp[:len(eddy_R)]

In [20]:
stereo_rotation = np.column_stack((eddy_L, eddy_R))
#write('turbulence/rotating_eddy.wav', SAMPLE_RATE, stereo_rotation)

In [21]:
np.max(stereo_rotation), np.max(turbulence_pt2)

(0.08557478893887381, 0.24509379700968228)

In [23]:
# Final assembly

turbulence_final = np.zeros((4 * 60 * SAMPLE_RATE, 2))

volMask = np.concatenate([
    np.linspace(0, 1, 10 * SAMPLE_RATE),
    np.ones(110 * SAMPLE_RATE)
])
turbulence_final[0 : len(turbulence_pt1)] += np.column_stack((volMask, volMask)) * turbulence_pt1

volMask = np.concatenate([
    np.ones(110 * SAMPLE_RATE),
    np.linspace(1, 0, 10 * SAMPLE_RATE),
])
turbulence_final[len(turbulence_pt1) : len(turbulence_pt1) + len(turbulence_pt2)] += np.column_stack((volMask, volMask)) * turbulence_pt2

volMask = np.concatenate([
    np.ones(110 * SAMPLE_RATE),
    np.linspace(1, 0, len(stereo_rotation) - 110 * SAMPLE_RATE)
])

turbulence_final[len(turbulence_pt1) : len(turbulence_pt1) + len(stereo_rotation)] += stereo_rotation * 20 * np.column_stack((volMask, volMask))


write('turbulence/turbulence.wav', SAMPLE_RATE, turbulence_final)